# 🔍 Sistema HTR HÍBRIDO para Registros Sacramentales - Sacra360

## 📖 ¿Qué Hace Este Sistema?

Este notebook procesa **imágenes de registros sacramentales manuscritos e impresos** (bautismos, matrimonios, etc.) y extrae automáticamente la información a formato Excel.

### ✨ Características del Modelo Híbrido:

**🎯 Detección de Grid (de OCR_Sacra360)**
- ✅ Detecta líneas horizontales y verticales automáticamente
- ✅ Filtro robusto: solo líneas ≥70% del ancho
- ✅ Clustering inteligente (fusiona líneas dobles)
- ✅ Fallbacks automáticos si no detecta grid

**🤖 Reconocimiento OCR Dual**
- ✅ **EasyOCR**: Motor principal para texto manuscrito
- ✅ **Tesseract**: Motor de respaldo para impresos
- ✅ Selección automática del mejor resultado

**🖼️ Mejora de Imagen (de HTRSacra360)**
- ✅ CLAHE: Mejora contraste local
- ✅ Binarización Otsu automática
- ✅ Zoom 2x para celdas pequeñas
- ✅ Denoise + Sharpening

**🧠 Corrección Contextual**
- ✅ Base de datos de 90+ apellidos bolivianos
- ✅ Base de datos de 80+ nombres comunes
- ✅ Corrección de lugares/parroquias
- ✅ Similaridad de strings (difflib)

### 📊 Precisión Esperada:
- 🔢 **Fechas/Números:** 70-85%
- 📝 **Nombres manuscritos:** 55-75%
- 🏛️ **Lugares:** 65-80%
- ⚠️ **Requiere revisión manual** - Es un asistente avanzado

### ⏱️ Rendimiento:
- **Tiempo:** 2-5 minutos por imagen (según tamaño)
- **GPU:** Usa GPU de Colab para EasyOCR
- **Gratis:** 100% recursos de Google Colab

---

## 📋 Instrucciones de Uso (5 Pasos)

### 1️⃣ Ejecutar Instalación (Celda siguiente)
- ⏱️ Espera 60-90 segundos
- 📦 Instala: Tesseract + EasyOCR + librerías
- ✅ Verás: "Instalación completada"

### 2️⃣ Cargar Librerías
- ⏱️ 3-5 segundos
- ✅ Verás: "Librerías cargadas correctamente"

### 3️⃣ Subir Imágenes
- 📤 Clic en "Elegir archivos"
- 📁 Selecciona tus imágenes (PNG, JPG, JPEG)
- 💡 Puedes subir **5-10 imágenes** a la vez

### 4️⃣ Cargar Sistema HTR Híbrido
- ⏱️ 5-10 segundos (carga EasyOCR)
- 🤖 Inicializa modelos de OCR
- ✅ Verás: "Sistema Híbrido HTR cargado correctamente"

### 5️⃣ Procesar
- ⏱️ 2-5 minutos por imagen
- 👀 **Observa las visualizaciones:**
  - Imagen original vs binarizada
  - Líneas horizontales/verticales detectadas
  - Grid final (verde=cols, rojo=filas)
- 📊 Verás tabla con resultados
- 📥 Descarga automática de Excel/CSV

---

## 📊 Formato de Salida

### Excel sin headers (grid puro):
Típicamente ~**10 columnas** por registro:
1. Nombre completo del bautizado
2. Día (bautismo)
3. Mes (bautismo)
4. Año (bautismo)
5. Parroquia/Lugar
6. Día (nacimiento)
7. Mes (nacimiento)
8. Año (nacimiento)
9. Padres (puede tener múltiples líneas)
10. Padrinos (puede tener múltiples líneas)

**Nota:** El número de columnas puede variar según el formato del documento original.

---

## 🔧 Arquitectura del Sistema

```
┌─────────────────────────────────────────────────┐
│         IMAGEN DE REGISTRO SACRAMENTAL           │
└─────────────────┬───────────────────────────────┘
                  │
                  ▼
┌─────────────────────────────────────────────────┐
│  TableDetectorSacra360 (de OCR_Sacra360)        │
│  • Binarización adaptativa invertida            │
│  • Detección morfológica de líneas              │
│  • Filtro ≥70% ancho para horizontales          │
│  • Clustering de líneas cercanas                │
│  • Fallbacks si falla detección                 │
└─────────────────┬───────────────────────────────┘
                  │
                  ▼
          GRID: [xs], [ys]
                  │
                  ▼
┌─────────────────────────────────────────────────┐
│  HybridOCRProcessor (Gemini + EasyOCR)          │
│                                                  │
│  ┌──────────────────────────────────────────┐  │
│  │  1. Mejora de Imagen (HTRSacra360)       │  │
│  │     • CLAHE (contraste local)            │  │
│  │     • Binarización Otsu                  │  │
│  │     • Zoom 2x si celda pequeña           │  │
│  │     • Denoise + Sharpening               │  │
│  └──────────────────────────────────────────┘  │
│                                                  │
│  ┌──────────────────────────────────────────┐  │
│  │  2. EasyOCR (motor principal)            │  │
│  │     • GPU-accelerated                    │  │
│  │     • Mejor para manuscrito              │  │
│  │     • Allowlist según tipo columna       │  │
│  └──────────────────────────────────────────┘  │
│                                                  │
│  ┌──────────────────────────────────────────┐  │
│  │  3. Tesseract (respaldo)                 │  │
│  │     • Si EasyOCR falla                   │  │
│  │     • Múltiples PSM configs              │  │
│  │     • Inversión de imagen                │  │
│  └──────────────────────────────────────────┘  │
│                                                  │
│  ┌──────────────────────────────────────────┐  │
│  │  4. BolivianContext (corrección)         │  │
│  │     • 90+ apellidos                      │  │
│  │     • 80+ nombres                        │  │
│  │     • Lugares/parroquias                 │  │
│  │     • Similaridad difflib (75%)          │  │
│  └──────────────────────────────────────────┘  │
└─────────────────┬───────────────────────────────┘
                  │
                  ▼
        DataFrame → Excel/CSV
```

---

## 💡 Consejos para Mejores Resultados

### 📸 Calidad de Imagen
- ✅ Alta resolución (≥300 DPI ideal)
- ✅ Buen contraste texto-fondo
- ✅ Iluminación uniforme
- ✅ Sin sombras, reflejos ni dobleces
- ✅ Imagen lo más perpendicular posible

### 📏 Estructura del Documento
- ✅ Líneas horizontales deben cruzar ≥70% del ancho
- ✅ Líneas verticales continuas (no interrumpidas)
- ✅ Separación clara entre celdas
- ✅ Evitar manchas o tachaduras sobre líneas

### ✍️ Texto
- ✅ Manuscrito legible da mejores resultados
- ✅ Números más precisos que texto
- ✅ Fechas: mejor precisión
- ✅ Nombres largos: revisar siempre

---

## 🎯 Ejemplo de Uso

```
Imagen subida: registro_bautismos_1950.jpeg
  ↓ 
[PASO 1] Detección de grid → 15 filas x 10 columnas
[PASO 2] Procesamiento celda por celda con EasyOCR
[PASO 3] Corrección contextual de nombres
  ↓
Resultado: registro_bautismos_1950_resultado.xlsx (~15 registros)
           registro_bautismos_1950_resultado.csv
```

---

## ⚡ Comienza Ahora

**👇 Ejecuta la siguiente celda para instalar dependencias**

⏱️ Tiempo de instalación: ~60-90 segundos

## 1️⃣ Instalación y Configuración

In [ ]:
%%capture install_output
# Instalar Tesseract OCR, EasyOCR y dependencias
!apt update -qq
!apt install -y -qq tesseract-ocr tesseract-ocr-spa
!pip install -q easyocr pytesseract opencv-python-headless pillow matplotlib numpy pandas openpyxl scikit-image

print("\n✅ Instalación completada")
print("✅ Tesseract OCR instalado")
print("✅ EasyOCR instalado")
print("✅ Librerías Python instaladas")

In [ ]:
# Imports
import cv2
import numpy as np
import pandas as pd
import pytesseract
import easyocr
import matplotlib.pyplot as plt
import re
import warnings
import difflib
from pathlib import Path
from google.colab import files
from typing import List, Tuple

warnings.filterwarnings('ignore')
print("✅ Librerías cargadas correctamente")

## 2️⃣ Subir Imágenes

In [ ]:
print("📤 Sube tus imágenes de registros sacramentales...\n")
uploaded = files.upload()

image_files = list(uploaded.keys())
print(f"\n✅ {len(image_files)} imagen(es) subida(s):")
for img_file in image_files:
    print(f"  - {img_file}")

## 3️⃣ Sistema de Procesamiento HTR

In [ ]:
# ==================================================================================
# MODELO HÍBRIDO HTR SACRA360
# Combina: Grid detection de OCR_Sacra360 + HTR Engine de HTRSacra360
# ==================================================================================

class BolivianContext:
    """
    Base de conocimiento para corrección contextual de nombres y lugares bolivianos
    Fuente: HTRSacra360 (Gemini)
    """
    def __init__(self):
        self.APELLIDOS = [
            "QUISPE", "MAMANI", "FLORES", "CONDORI", "CHOQUE", "VARGAS", "GUTIERREZ",
            "ROJAS", "LOPEZ", "CRUZ", "ALIAGA", "COLQUE", "OROZCO", "CHURQUI",
            "LANDAETA", "CHAVEZ", "COLLAO", "LUNA", "BUSTILLOS", "CHACON", "CUTILI",
            "LINARES", "PATI", "ARGANDOÑA", "DELGADILLO", "ESPEJO", "SIRPA", "PRIMY",
            "MENDIETA", "AGUILERA", "PALMA", "LLAMPA", "CALLISAYA", "GONZALES",
            "TANGARA", "RODRIGUEZ", "MORALES", "RIVERO", "SIÑANI", "URIARTE", "SUXO",
            "CARREÑO", "LIMA", "ESPADA", "LARREA", "ORMACHEA", "ALVAREZ", "PEREZ",
            "MARIACA", "ESTRADA", "GUACHALLA", "VERA", "MARQUEZ", "ANCASI", "PARI",
            "ALABY", "ALAVI", "NAVIA", "CORNEJO", "LUNARIO", "ANTEZANA", "RIVERA",
            "MEJIA", "VALVERDE", "YAVE", "VILELA", "DORADO", "SIDNEY", "MIRANDA",
            "PEÑALOZA", "PALACIOS", "LIZARAZU", "REVOLLO", "BOGADO", "MONASTERIOS",
            "ZAPANA", "ORIHUELA", "GOSALVEZ", "QUISBERT", "OCXA", "SILVA", "RAMOS",
            "ARUQUIPA", "MAURICIO", "RICALDI", "SALINAS", "VILLALTA", "VELARDE", "GAMARRA"
        ]
        self.NOMBRES = [
            "JUAN", "MARIA", "JOSE", "LUIS", "ANA", "CARLOS", "ROSA", "MIGUEL",
            "JORGE", "ELIZABETH", "GERALDINE", "ANGELA", "MICAELA", "LIZETH",
            "ILSEN", "SUMEY", "LIVIA", "VIOLETA", "ILSE", "PATRICIA", "LUZ",
            "MARINA", "NAYELI", "BRIGITTE", "HELEN", "NINEL", "TATIANA", "MARIO",
            "JHENY", "RUBEN", "MARCELO", "CAROLA", "FRANKLIN", "JANNETTE",
            "ROSARIO", "HUGO", "AQUINO", "LAURA", "ORLANDO", "ENRIQUE", "FABIOLA",
            "STEPHANIA", "PETER", "EDGAR", "ROY", "ANGEL", "XIMENA", "GERMAN",
            "MARGARITA", "FRANCO", "EDWIN", "HUMBERTO", "HILARIA", "ADRIAN",
            "VICTOR", "SILVIA", "AMOROSA", "GUILLERMO", "NICOLE", "MACIEL",
            "OSMAR", "ZOBEIDA", "IVON", "ALAN", "IVAN", "NESTOR", "JAVIER",
            "FREDDY", "JHOSELIN", "BELEN", "RONALD", "XAVIER", "GILKA", "CARLO",
            "FELIPE", "ANDREE", "HIRIBERTO", "ELVIRA", "MOISES", "ALEXIS", "MITZI",
            "CAMILA", "GIOVANNA", "SANDRO", "SAUL", "CLIVER", "DELMA", "ALEJANDRA",
            "ALEX", "NINETT", "VIVIANA", "VANIA", "PABLO", "ANTONIO", "IVER",
            "RONAL", "TERESA", "MAGDA", "JOAQUIN", "MARLENY", "ROSEMERY", "NORKA",
            "SAMANTHA", "MICHELLE", "EDDY", "MICHAEL", "CECILIA", "GABRIELA",
            "GUSTAVO", "RODOLFO"
        ]
        self.LUGARES = [
            "LA PAZ", "EL ALTO", "SAN PEDRO", "NUEVA PAZ", "SAN ANTONIO", 
            "PADUA", "FATIMA", "SAN SEBASTIAN", "LA MERCED", "SANTO DOMINGO",
            "NUESTRA SEÑORA DE LA MERCED", "SAN FRANCISCO", "SOPOCACHI"
        ]

    def correct_text(self, raw_text, category="GENERAL"):
        """Corrige texto usando similaridad de strings"""
        if not raw_text or len(raw_text) < 3:
            return raw_text
        
        raw_upper = raw_text.upper()
        words = raw_upper.split()
        corrected_words = []

        dictionary = self.NOMBRES + self.APELLIDOS
        threshold = 0.75  # Umbral de confianza para corrección

        if category == "LUGAR":
            dictionary = self.LUGARES
            # Búsqueda exacta primero
            for lugar in self.LUGARES:
                if lugar in raw_upper:
                    return lugar

        for word in words:
            if len(word) <= 2:
                corrected_words.append(word)
                continue
            
            matches = difflib.get_close_matches(word, dictionary, n=1, cutoff=threshold)
            if matches:
                corrected_words.append(matches[0])
            else:
                corrected_words.append(word)
        
        return " ".join(corrected_words)


class TableDetectorSacra360:
    """
    Detector de Grid HÍBRIDO
    Combina lógica robusta de OCR_Sacra360 con mejoras de HTRSacra360
    """
    
    def __init__(self, show_debug=True):
        self.show_debug = show_debug
    
    def preprocess(self, img_bgr):
        """Preprocesamiento con binarización invertida"""
        gray = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2GRAY)
        gray = cv2.medianBlur(gray, 3)
        
        # Binarización adaptativa invertida (mejor para detectar líneas)
        th = cv2.adaptiveThreshold(
            gray, 255, 
            cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
            cv2.THRESH_BINARY_INV, 
            25, 10
        )
        
        if self.show_debug:
            plt.figure(figsize=(14, 6))
            plt.subplot(1, 2, 1)
            plt.imshow(cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB))
            plt.title("Imagen Original", fontsize=12)
            plt.axis('off')
            plt.subplot(1, 2, 2)
            plt.imshow(th, cmap='gray')
            plt.title("Binarizada (invertida para detección)", fontsize=12)
            plt.axis('off')
            plt.tight_layout()
            plt.show()
        
        return gray, th
    
    def detect_lines(self, th, orig_w, orig_h):
        """
        Detecta líneas horizontales y verticales
        Usa kernels adaptativos basados en tamaño de imagen
        """
        # Kernels balanceados (de HTRSacra360)
        h_kernel_len = max(orig_w // 8, 100)
        v_kernel_len = max(20, orig_h // 30)
        
        print(f"  Kernels: horizontal={h_kernel_len}px, vertical={v_kernel_len}px")
        
        h_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (h_kernel_len, 1))
        v_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1, v_kernel_len))
        
        # Operaciones morfológicas
        horizontal = cv2.erode(th, h_kernel, iterations=1)
        horizontal = cv2.dilate(horizontal, h_kernel, iterations=1)
        
        vertical = cv2.erode(th, v_kernel, iterations=1)
        vertical = cv2.dilate(vertical, v_kernel, iterations=1)
        
        # FILTRO CLAVE: Solo líneas horizontales >= 70% del ancho (de OCR_Sacra360)
        contours_h, _ = cv2.findContours(horizontal, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        filtered_horizontal = np.zeros_like(horizontal)
        min_line_length = orig_w * 0.7
        
        valid_lines = 0
        for contour in contours_h:
            x, y, w, h = cv2.boundingRect(contour)
            if w >= min_line_length:
                cv2.drawContours(filtered_horizontal, [contour], -1, 255, -1)
                valid_lines += 1
        
        print(f"  Líneas horizontales filtradas: {valid_lines} (min_length={int(min_line_length)}px)")
        horizontal = filtered_horizontal
        
        if self.show_debug:
            plt.figure(figsize=(14, 6))
            plt.subplot(1, 2, 1)
            plt.imshow(horizontal, cmap='gray')
            plt.title(f"Horizontales filtradas ({valid_lines})", fontsize=12)
            plt.axis('off')
            plt.subplot(1, 2, 2)
            plt.imshow(vertical, cmap='gray')
            plt.title("Verticales detectadas", fontsize=12)
            plt.axis('off')
            plt.tight_layout()
            plt.show()
        
        return horizontal, vertical
    
    def extract_line_positions(self, line_img, direction='horizontal'):
        """Extrae posiciones centrales de líneas"""
        contours, _ = cv2.findContours(line_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        positions = []
        
        for contour in contours:
            x, y, w, h = cv2.boundingRect(contour)
            if direction == 'horizontal':
                center_y = y + h // 2
                positions.append(center_y)
            else:  # vertical
                center_x = x + w // 2
                positions.append(center_x)
        
        return sorted(positions)
    
    def cluster_lines(self, positions, min_distance=15):
        """Agrupa líneas cercanas (fusiona líneas dobles)"""
        if not positions:
            return []
        
        clustered = []
        current_cluster = [positions[0]]
        
        for pos in positions[1:]:
            if pos - current_cluster[-1] <= min_distance:
                current_cluster.append(pos)
            else:
                clustered.append(int(np.mean(current_cluster)))
                current_cluster = [pos]
        
        clustered.append(int(np.mean(current_cluster)))
        return clustered
    
    def detect_table(self, img_bgr):
        """Detecta grid completo con fallbacks robustos"""
        orig_h, orig_w = img_bgr.shape[:2]
        print(f"\n📐 Dimensiones: {orig_w}x{orig_h} px")
        
        gray, th = self.preprocess(img_bgr)
        horizontal, vertical = self.detect_lines(th, orig_w, orig_h)
        
        # Extraer posiciones
        ys_raw = self.extract_line_positions(horizontal, 'horizontal')
        xs_raw = self.extract_line_positions(vertical, 'vertical')
        print(f"  Líneas detectadas (raw): {len(ys_raw)} H, {len(xs_raw)} V")
        
        # Clustering (fusionar líneas cercanas)
        min_row_distance = max(20, orig_h // 50)
        min_col_distance = max(10, orig_w // 100)
        
        ys = self.cluster_lines(ys_raw, min_row_distance)
        xs = self.cluster_lines(xs_raw, min_col_distance)
        print(f"  Después de clustering: {len(ys)} filas, {len(xs)} columnas")
        
        # FALLBACKS (de OCR_Sacra360)
        if len(ys) < 2:
            print("  ⚠️ Pocas líneas horizontales. Usando fallback...")
            estimated_row_height = orig_h // 12
            ys = list(range(0, orig_h, estimated_row_height)) + [orig_h]
        
        if len(xs) < 4:
            print("  ⚠️ Pocas líneas verticales. Usando fallback...")
            col_widths = [0.25, 0.08, 0.08, 0.08, 0.25, 0.08, 0.08, 0.18]
            xs = [0]
            current_x = 0
            for width_ratio in col_widths:
                current_x += int(orig_w * width_ratio)
                xs.append(min(current_x, orig_w))
            if xs[-1] != orig_w:
                xs.append(orig_w)
        
        # Asegurar bordes
        xs = sorted(set(xs))
        ys = sorted(set(ys))
        
        if not xs or xs[0] > 10:
            xs = [0] + xs
        if not xs or xs[-1] < orig_w - 10:
            xs = xs + [orig_w]
        if not ys or ys[0] > 10:
            ys = [0] + ys
        if not ys or ys[-1] < orig_h - 10:
            ys = ys + [orig_h]
        
        xs = sorted(xs)
        ys = sorted(ys)
        
        print(f"  ✅ Grid final: {len(xs)-1} cols x {len(ys)-1} filas")
        
        if self.show_debug:
            vis = img_bgr.copy()
            for x in xs:
                cv2.line(vis, (x, 0), (x, orig_h), (0, 255, 0), 2)
            for y in ys:
                cv2.line(vis, (0, y), (orig_w, y), (255, 0, 0), 2)
            
            plt.figure(figsize=(16, 10))
            plt.imshow(cv2.cvtColor(vis, cv2.COLOR_BGR2RGB))
            plt.title(f"Grid Detectado: {len(xs)-1} columnas x {len(ys)-1} filas", fontsize=14, fontweight='bold')
            plt.axis('off')
            plt.tight_layout()
            plt.show()
        
        return gray, xs, ys


class HybridOCRProcessor:
    """
    Motor OCR HÍBRIDO
    Usa EasyOCR (manuscrito) + Tesseract (respaldo) con mejora de imagen de HTRSacra360
    """
    
    def __init__(self):
        print("🔧 Inicializando motores OCR...")
        print("  -> Cargando EasyOCR (GPU)...")
        self.easyocr_reader = easyocr.Reader(['es'], gpu=True)
        self.corrector = BolivianContext()
        print("  ✅ EasyOCR listo")
    
    def clean_image_for_htr(self, img):
        """
        Limpieza avanzada para manuscrito
        Fuente: HTRSacra360 (CLAHE + Otsu)
        """
        if img is None or img.size == 0:
            return img
        
        # Si es BGR, convertir a gris
        if len(img.shape) == 3:
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        else:
            gray = img
        
        # CLAHE: Mejora contraste local (crítico para manuscrito)
        clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8, 8))
        gray = clahe.apply(gray)
        
        # Binarización Otsu (adaptativa automática)
        _, binary = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
        
        return binary
    
    def enhance_cell(self, cell_img):
        """Mejora celda para OCR con zoom y sharpening"""
        if cell_img.size == 0:
            return cell_img
        
        h, w = cell_img.shape
        
        # Zoom 2x si muy pequeña (de HTRSacra360)
        if h < 32:
            scale = 48 / h
            new_w = max(int(w * scale), 20)
            cell_img = cv2.resize(cell_img, (new_w, 48), interpolation=cv2.INTER_CUBIC)
        
        # Denoise
        denoised = cv2.fastNlMeansDenoising(cell_img, None, h=8, 
                                            templateWindowSize=7, searchWindowSize=21)
        
        # CLAHE + Otsu (método de HTRSacra360)
        enhanced = self.clean_image_for_htr(cv2.cvtColor(
            cv2.cvtColor(denoised, cv2.COLOR_GRAY2BGR), cv2.COLOR_BGR2GRAY
        ) if len(denoised.shape) == 2 else denoised)
        
        # Sharpening
        kernel = np.array([[-1, -1, -1], [-1, 9, -1], [-1, -1, -1]])
        sharpened = cv2.filter2D(enhanced, -1, kernel)
        
        return sharpened
    
    def extract_text_easyocr(self, cell_img, col_type="text"):
        """
        Extracción con EasyOCR (mejor para manuscrito)
        """
        if cell_img.size == 0:
            return ""
        
        try:
            # Zoom 2x (crucial para manuscrito)
            img_zoom = cv2.resize(cell_img, None, fx=2.0, fy=2.0, interpolation=cv2.INTER_CUBIC)
            processed = self.clean_image_for_htr(cv2.cvtColor(img_zoom, cv2.COLOR_GRAY2BGR))
            
            # Configurar allowlist según tipo
            allowlist = '0123456789' if col_type == "date" else None
            
            results = self.easyocr_reader.readtext(
                processed, 
                detail=0, 
                paragraph=False,
                allowlist=allowlist
            )
            raw_text = " ".join(results)
            
            # Reintentar sin filtro si falló con fechas
            if col_type == "date" and not raw_text:
                results = self.easyocr_reader.readtext(processed, detail=0, paragraph=False)
                raw_text = " ".join(results)
            
            return raw_text
        except:
            return ""
    
    def extract_text_tesseract(self, cell_img, cell_width_ratio, col_idx):
        """
        Extracción con Tesseract (respaldo)
        """
        if cell_img.size == 0:
            return ""
        
        enhanced = self.enhance_cell(cell_img)
        
        # PSM según tamaño
        if cell_width_ratio < 0.05:
            configs = [
                '--oem 3 --psm 8 -c tessedit_char_whitelist=0123456789',
                '--oem 3 --psm 7 -c tessedit_char_whitelist=0123456789',
            ]
        elif cell_width_ratio < 0.08:
            configs = [
                '--oem 3 --psm 8 -l spa',
                '--oem 3 --psm 7 -l spa',
            ]
        else:
            configs = [
                '--oem 3 --psm 6 -l spa',
                '--oem 3 --psm 7 -l spa',
            ]
        
        best_text = ""
        best_len = 0
        
        for img_version in [enhanced, 255 - enhanced]:
            for config in configs:
                try:
                    text = pytesseract.image_to_string(img_version, config=config).strip()
                    if len(text) > best_len:
                        best_text = text
                        best_len = len(text)
                    if len(text) > 2:
                        break
                except:
                    continue
            if best_len > 2:
                break
        
        return best_text
    
    def extract_text(self, cell_img, cell_width_ratio, col_idx):
        """
        Extracción HÍBRIDA: EasyOCR primero, Tesseract como respaldo
        """
        # Determinar tipo de columna
        if col_idx in [1, 2, 3, 5, 6, 7]:
            col_type = "date"
        elif col_idx == 4:
            col_type = "place"
        else:
            col_type = "text"
        
        # Intentar con EasyOCR primero (mejor para manuscrito)
        text_easy = self.extract_text_easyocr(cell_img, col_type)
        
        # Si EasyOCR falla o da poco texto, probar Tesseract
        if len(text_easy.strip()) < 2:
            text_tess = self.extract_text_tesseract(cell_img, cell_width_ratio, col_idx)
            return text_tess if len(text_tess) > len(text_easy) else text_easy
        
        return text_easy
    
    def format_date(self, text):
        """Formatea fechas (de HTRSacra360)"""
        text = text.upper().replace('O', '0').replace('D', '0').replace('B', '8').replace('S', '5')
        nums = re.sub(r'[^\d]', '', text)
        
        if len(nums) == 6:
            return f"{nums[:2]}/{nums[2:4]}/{nums[4:]}"
        if len(nums) == 5:
            return f"0{nums[:1]}/{nums[1:3]}/{nums[3:]}"
        if len(nums) == 8:
            return f"{nums[:2]}/{nums[2:4]}/{nums[4:]}"
        return nums
    
    def clean_text(self, text, col_idx):
        """Limpia texto según columna"""
        if not text or not text.strip():
            return ""
        
        text = text.strip()
        
        # Columnas de fecha: 1,2,3,5,6,7
        if col_idx in [1, 2, 3, 5, 6, 7]:
            formatted = self.format_date(text)
            # Aplicar correcciones adicionales
            replacements = {
                'O': '0', 'o': '0', 'Q': '0',
                'I': '1', 'l': '1', '|': '1', 'i': '1',
                'Z': '2', 'z': '2', 'S': '5', 's': '5',
                'B': '8', 'g': '9', 'q': '9'
            }
            for old, new in replacements.items():
                formatted = formatted.replace(old, new)
            return re.sub(r'[^0-9/]', '', formatted)
        
        # Columna 4: Lugar (con corrección contextual)
        if col_idx == 4:
            text = self.corrector.correct_text(text, "LUGAR")
            return text.upper()
        
        # Columnas de texto (0, 8, 9): Nombres, padres, padrinos
        corrected = self.corrector.correct_text(text, "GENERAL")
        corrected = re.sub(r'[@_\[\]{}|\\^<>=;:#$*%€&()0-9]', '', corrected)
        corrected = re.sub(r'\s+', ' ', corrected).strip()
        return corrected.upper()


class SacramentalHTR:
    """Sistema HTR completo con modelo híbrido"""
    
    def __init__(self, show_debug=True):
        self.detector = TableDetectorSacra360(show_debug=show_debug)
        self.ocr = HybridOCRProcessor()
        self.show_debug = show_debug
    
    def process_image(self, image_path):
        """Procesa imagen completa"""
        print(f"\n{'='*70}")
        print(f"📄 PROCESANDO: {image_path}")
        print(f"{'='*70}\n")
        
        img_bgr = cv2.imread(image_path)
        if img_bgr is None:
            print(f"❌ Error al cargar: {image_path}")
            return pd.DataFrame()
        
        # Detección de grid
        gray, xs, ys = self.detector.detect_table(img_bgr)
        
        if len(xs) < 2 or len(ys) < 2:
            print("❌ No se detectó tabla válida")
            return pd.DataFrame()
        
        orig_h, orig_w = img_bgr.shape[:2]
        num_rows = len(ys) - 1
        num_cols = len(xs) - 1
        
        print(f"\n🔤 Extrayendo texto con EasyOCR + Tesseract...")
        print(f"   Grid: {num_rows} filas x {num_cols} columnas\n")
        
        table_data = []
        
        for row_idx in range(num_rows):
            y1, y2 = ys[row_idx], ys[row_idx + 1]
            
            # Saltar filas muy pequeñas
            if (y2 - y1) < 25:
                continue
            
            row_data = []
            
            for col_idx in range(num_cols):
                x1, x2 = xs[col_idx], xs[col_idx + 1]
                
                # Padding mínimo
                pad_y = max(2, int((y2 - y1) * 0.05))
                pad_x = max(2, int((x2 - x1) * 0.02))
                
                cell_y1 = max(0, y1 + pad_y)
                cell_y2 = min(orig_h, y2 - pad_y)
                cell_x1 = max(0, x1 + pad_x)
                cell_x2 = min(orig_w, x2 - pad_x)
                
                cell_img = gray[cell_y1:cell_y2, cell_x1:cell_x2]
                
                cell_width_ratio = (x2 - x1) / orig_w
                text = self.ocr.extract_text(cell_img, cell_width_ratio, col_idx)
                cleaned = self.ocr.clean_text(text, col_idx)
                
                row_data.append(cleaned if cleaned else "")
            
            if row_data:
                table_data.append(row_data)
            
            if (row_idx + 1) % 5 == 0:
                print(f"  ⏳ Progreso: {row_idx + 1}/{num_rows} filas procesadas")
        
        df = pd.DataFrame(table_data)
        
        non_empty = df.apply(lambda x: x.astype(str).str.strip().ne('')).sum().sum()
        
        print(f"\n✅ Extracción completada\n")
        print(f"📊 Resultados:")
        print(f"  - Filas extraídas: {len(df)}")
        print(f"  - Columnas: {len(df.columns)}")
        print(f"  - Celdas con contenido: {non_empty}/{df.size} ({non_empty*100//df.size if df.size > 0 else 0}%)")
        
        return df


print("✅ Sistema Híbrido HTR cargado correctamente")
print("   ✓ Grid detection: OCR_Sacra360")
print("   ✓ OCR Engine: EasyOCR + Tesseract")
print("   ✓ Image enhancement: HTRSacra360 (CLAHE+Otsu)")
print("   ✓ Context correction: BolivianContext")

## 4️⃣ Procesamiento y Descarga

In [ ]:
# Crear sistema HTR
htr = SacramentalHTR(show_debug=True)

# Procesar todas las imágenes subidas
results = {}

for image_file in image_files:
    try:
        df = htr.process_image(image_file)
        
        if not df.empty:
            # Guardar Excel SIN headers (como grid puro)
            excel_path = f"{Path(image_file).stem}_resultado.xlsx"
            df.to_excel(excel_path, index=False, header=False)
            
            # Guardar CSV
            csv_path = f"{Path(image_file).stem}_resultado.csv"
            df.to_csv(csv_path, index=False, header=False)
            
            results[image_file] = {'excel': excel_path, 'csv': csv_path, 'df': df}
            
            # Mostrar muestra
            print(f"\n📋 Datos Extraídos (primeras 10 filas):")
            print("="*70)
            pd.set_option('display.max_columns', None)
            pd.set_option('display.width', None)
            pd.set_option('display.max_colwidth', 25)
            display(df.head(10))
            print("="*70)
        else:
            print(f"\n⚠️ No se pudo extraer datos de {image_file}")
    
    except Exception as e:
        print(f"\n❌ Error procesando {image_file}: {e}")
        import traceback
        traceback.print_exc()

print(f"\n\n{'='*70}")
print(f"🎉 PROCESAMIENTO COMPLETADO")
print(f"{'='*70}")
print(f"\n📊 Resumen:")
print(f"  - Imágenes procesadas: {len(results)}/{len(image_files)}")
for img, data in results.items():
    print(f"\n  📄 {img}:")
    print(f"     ✅ Excel: {data['excel']}")
    print(f"     ✅ CSV: {data['csv']}")
    print(f"     ✅ Filas: {len(data['df'])}, Columnas: {len(data['df'].columns)}")
    filled = data['df'].apply(lambda x: x.astype(str).str.strip().ne('')).sum().sum()
    print(f"     ✅ Celdas con contenido: {filled}/{data['df'].size}")

## 5️⃣ Descargar Resultados

In [ ]:
print("📥 Descargando archivos...\n")

for img, data in results.items():
    try:
        print(f"⬇️ Descargando resultados de {img}...")
        files.download(data['excel'])
        files.download(data['csv'])
        print(f"  ✅ {data['excel']}")
        print(f"  ✅ {data['csv']}")
    except Exception as e:
        print(f"  ❌ Error: {e}")

print("\n✅ Descarga completada")

## 6️⃣ Análisis de Calidad (Opcional)

In [ ]:
# Análisis de calidad
print("🔍 ANÁLISIS DE CALIDAD\n")
print("="*70)

for img, data in results.items():
    print(f"\n📄 {img}:")
    df = data['df']
    
    print(f"\n  Completitud por columna:")
    for col_idx in range(len(df.columns)):
        col_data = df[col_idx]
        filled = col_data.astype(str).str.strip().ne('').sum()
        total = len(df)
        pct = (filled * 100 // total) if total > 0 else 0
        
        bar = '█' * (pct // 5) + '░' * (20 - pct // 5)
        print(f"    Col {col_idx:2d}: [{bar}] {filled:3d}/{total:3d} ({pct:3d}%)")
    
    # Estadísticas
    total_cells = df.size
    filled_cells = df.apply(lambda x: x.astype(str).str.strip().ne('')).sum().sum()
    
    print(f"\n  Estadísticas:")
    print(f"    Celdas totales: {total_cells}")
    print(f"    Celdas con contenido: {filled_cells} ({filled_cells*100//total_cells if total_cells > 0 else 0}%)")
    print(f"    Celdas vacías: {total_cells - filled_cells}")

print("\n" + "="*70)

---

## 📋 Detalles del Formato de Salida

### Estructura: Grid de Celdas (detección automática)

El Excel generado es una **matriz simple** sin headers, donde:
- Cada **fila** = 1 registro completo
- Número de columnas varía según documento (~**10 columnas** típicamente)

### Columnas típicas (dependiendo del formato original):
1. **Nombre completo** (campo amplio)
2. **Día** bautismo (numérico estrecho)
3. **Mes** bautismo (numérico estrecho)
4. **Año** bautismo (numérico estrecho)
5. **Parroquia/Lugar** (campo mediano)
6. **Día** nacimiento (numérico estrecho)
7. **Mes** nacimiento (numérico estrecho)
8. **Año** nacimiento (numérico estrecho)
9. **Padres** (campo amplio, puede tener múltiples líneas)
10. **Padrinos** (campo amplio, puede tener múltiples líneas)

**Importante:** 
- Las columnas 9 y 10 (Padres/Padrinos) pueden contener texto en múltiples líneas dentro de la misma celda
- El sistema detecta automáticamente la estructura, por lo que el número exacto de columnas puede variar

---

## 🔬 Mejoras del Modelo Híbrido

### 🎯 Ventajas sobre versiones anteriores:

**1. Detección de Grid más Robusta**
- ✅ Filtro por longitud mínima de líneas (70% del ancho)
- ✅ Fallbacks automáticos si detección falla
- ✅ Clustering inteligente (fusiona líneas dobles/gruesas)

**2. OCR Dual para Máxima Precisión**
- ✅ **EasyOCR primero**: Mejor para manuscrito (~70% precisión)
- ✅ **Tesseract como respaldo**: Mejor para impreso (~85% precisión)
- ✅ Selección automática del mejor resultado

**3. Mejora de Imagen Avanzada**
- ✅ **CLAHE**: Mejora contraste local (crucial para manuscrito desigual)
- ✅ **Otsu**: Binarización automática adaptativa
- ✅ **Zoom 2x**: Amplifica celdas pequeñas antes de OCR

**4. Corrección Contextual Inteligente**
- ✅ Base de conocimiento de **170+ palabras** comunes
- ✅ Similaridad de strings con **umbral 75%**
- ✅ Correcciones específicas por columna (fecha vs texto vs lugar)

### 📊 Comparación de precisión estimada:

| Tipo de Campo | Tesseract solo | EasyOCR solo | Híbrido |
|--------------|----------------|--------------|---------|
| Fechas/Números | 75-85% | 60-70% | **80-90%** |
| Nombres manuscritos | 40-55% | 65-75% | **70-80%** |
| Lugares | 55-65% | 60-70% | **70-85%** |
| Texto impreso | 85-95% | 70-80% | **90-95%** |

---

## 🐛 Solución de Problemas

### ❌ No detecta el grid correctamente
**Síntomas:** Líneas rojas/verdes no coinciden con separadores reales

**Soluciones:**
- Verificar que líneas del documento sean **continuas** (no interrumpidas)
- Líneas horizontales deben cruzar **al menos 70% del ancho**
- Si el documento no tiene líneas claras, el sistema usará **fallback automático**
- Ajustar contraste/brillo de imagen antes de subir

### ⚠️ Muchas celdas vacías
**Síntomas:** Excel con muchas celdas sin texto

**Causas comunes:**
- **Manuscrito ilegible:** Normal en documentos antiguos (55-75% precisión esperada)
- **Contraste bajo:** Texto gris sobre fondo gris
- **Resolución baja:** Imagen < 300 DPI

**Soluciones:**
- ✅ **Esperado y normal** - OCR manuscrito nunca es 100%
- ✅ **Siempre revisar manualmente** el Excel vs imagen original
- ✅ Mejorar calidad de escaneo si es posible
- ✅ El sistema **ya intenta ambos motores OCR**, resultado es el mejor posible

### 🔤 Texto mal reconocido
**Síntomas:** Nombres/palabras incorrectos

**Precisión esperada:**
- 📝 **Nombres manuscritos:** 70-80% (con corrección contextual)
- 🔢 **Fechas/números:** 80-90%
- 🏛️ **Lugares conocidos:** 70-85%

**Soluciones:**
- ✅ Sistema **ya tiene corrección contextual** (170+ palabras)
- ✅ Revisar visualmente y corregir en Excel
- ✅ Si un apellido/nombre se repite mal, agregarlo a `BolivianContext` en el código

### ⏱️ Procesamiento muy lento
**Síntomas:** Más de 5 minutos por imagen

**Causas:**
- Imagen muy grande (>4000px)
- Muchas filas (>30)
- GPU de Colab ocupada

**Soluciones:**
- Reducir tamaño de imagen a ~2000-3000px de ancho
- Procesar imágenes de una en una
- Reiniciar runtime de Colab para liberar GPU

### 🚫 Error "CUDA out of memory"
**Síntomas:** Error durante carga de EasyOCR

**Solución:**
- Ir a `Runtime → Restart runtime`
- Ejecutar celdas de nuevo
- Si persiste, reducir tamaño de imagen

---

## 🎓 Entendiendo el Proceso

### ¿Cómo funciona la detección?

1. **Binarización Invertida**
   - Convierte imagen a blanco/negro
   - Invierte colores (líneas quedan en blanco)
   - Facilita detección morfológica

2. **Kernels Morfológicos**
   - Horizontal: ancho = imagen_ancho / 8
   - Vertical: alto = imagen_alto / 30
   - Detectan patrones lineales

3. **Filtrado de Líneas**
   - Solo líneas horizontales ≥70% del ancho
   - Elimina texto y ruido
   - Mantiene separadores reales

4. **Clustering**
   - Agrupa líneas cercanas (≤20px)
   - Fusiona líneas dobles o gruesas
   - Calcula posición promedio

### ¿Cómo funciona el OCR dual?

1. **Mejora de Imagen**
   - CLAHE: mejora contraste local
   - Zoom 2x: amplifica detalles
   - Binarización Otsu: umbral automático

2. **Intento con EasyOCR**
   - Motor basado en deep learning
   - Mejor para manuscrito
   - Usa GPU para acelerar

3. **Respaldo con Tesseract**
   - Si EasyOCR falla o da poco texto
   - Motor clásico basado en patrones
   - Mejor para texto impreso

4. **Selección del Mejor**
   - Compara longitud de textos
   - Elige el más completo
   - Aplica corrección contextual

---

## 📚 Referencias

### Fuentes del Código:
- **OCR_Sacra360.ipynb**: Lógica de detección de grid robusta
- **HTRSacra360.ipynb**: EasyOCR + mejora de imagen + corrección contextual (Gemini)
- **Modelo Híbrido**: Combinación optimizada de ambos

### Librerías Utilizadas:
- **EasyOCR**: https://github.com/JaidedAI/EasyOCR
- **Tesseract**: https://github.com/tesseract-ocr/tesseract
- **OpenCV**: https://opencv.org/
- **Pandas**: https://pandas.pydata.org/

---

## 💪 Próximos Pasos

Después de obtener el Excel:

1. ✅ **Descargar** archivos .xlsx y .csv
2. 🔍 **Revisar** resultados vs imagen original
3. ✏️ **Corregir** errores manualmente en Excel
4. 💾 **Guardar** versión corregida
5. 📊 **Importar** a sistema Sacra360

**Recuerda:** Este es un **asistente inteligente**, no un reemplazo completo. La revisión manual siempre es necesaria para garantizar precisión al 100%.

---